# Johnson's Algorithm: Complete Beginner-to-Expert Guide

## 🌟 What is Johnson's Algorithm?

**Imagine you're a logistics company that needs to find the shortest routes between ALL pairs of cities in your delivery network.** Some routes have tolls (positive costs), some have government subsidies (negative costs), but you want to use the fastest GPS algorithm (Dijkstra) which only works with positive costs. Johnson's algorithm is like a clever accountant who "adjusts the books" to make all costs positive while keeping the relative route rankings exactly the same!

**Simple Definition:** Johnson's algorithm finds **shortest paths between ALL pairs of vertices** in a weighted graph (including negative weights) by cleverly transforming the graph to have non-negative weights, then using Dijkstra efficiently.

### 🏢 Real-World Analogy: Corporate Restructuring

Think of Johnson's algorithm as a financial restructuring consultant:

1. **You have a business network** with various costs and credits between departments
2. **Some departments give rebates** (negative edge weights)
3. **You want to use efficient accounting software** (Dijkstra) that only handles positive numbers
4. **The consultant adjusts all the books** using a "potential function" to make everything positive
5. **The relative costs stay the same** - cheapest routes remain cheapest!

```
Example: Department Cost Network
Sales ——$50——→ Marketing ——$30——→ IT
  ↑                              ↓
$(-20)                          $10
  ↑                              ↓
HR  ←——————$40—————————————— Finance

Johnson's magic: Convert all to positive while preserving shortest paths!
```

## 🎯 Why Do We Need Johnson's Algorithm?

### The All-Pairs Shortest Path (APSP) Problem

**The Challenge:** We need shortest distances between **every pair** of vertices, not just from one source.

### Why Existing Algorithms Fall Short

```
Problem: Mixed positive/negative weights in sparse graph

Option 1: Dijkstra × V times
❌ Fails with negative weights

Option 2: Bellman-Ford × V times  
❌ Too slow: O(V²E)

Option 3: Floyd-Warshall
❌ O(V³) - inefficient for sparse graphs

✅ Johnson's Algorithm
Handles negatives + Fast O(V²log V + VE) for sparse graphs!
```

## 🔍 How Does Johnson's Algorithm Work? (Simple Explanation)

### The Genius Insight

**"Transform the graph to make all edge weights non-negative while preserving shortest path relationships, then use fast Dijkstra!"**

### The Magic Transformation

Johnson's uses a **potential function** h(v) for each vertex:
- **Reweight each edge:** `new_weight(u,v) = old_weight(u,v) + h(u) - h(v)`
- **Key property:** All paths between same endpoints change by the same amount!
- **Result:** Shortest paths are preserved, but now all weights can be made non-negative

### Step-by-Step Process:

1. **Add a super source** connected to all vertices with 0-weight edges
2. **Run Bellman-Ford** from super source to get potential function h(v)
3. **Check for negative cycles** - if found, report and stop
4. **Reweight all edges** using potential function
5. **Run Dijkstra from each vertex** in the reweighted graph
6. **Convert distances back** to original graph using potential function

## 📚 Algorithm Family Context

| Problem Type | Algorithm | Time Complexity | Best For |
|-------------|-----------|-----------------|----------|
| Single Source, unweighted | BFS | O(V + E) | Simple graphs |
| Single Source, non-negative | Dijkstra | O(V log V + E) | GPS, routing |
| Single Source, any weights | Bellman-Ford | O(VE) | Negative weights |
| **All Pairs, any weights, dense** | **Floyd-Warshall** | **O(V³)** | **Dense graphs** |
| **All Pairs, any weights, sparse** | **Johnson's** | **O(V²log V + VE)** | **Sparse graphs** |

**Key Point:** Johnson's is optimal for **all-pairs shortest paths** in **sparse graphs** with **negative edge weights**.

## 🔧 The Algorithm in Detail

### Phase 1: Create Super Source and Check for Negative Cycles

**Step 1:** Add vertex `s` with 0-weight edges to all original vertices
```
Original graph + Super source s:
     s
   ↙ ↓ ↘ (all weight 0)
  A  B  C
```

**Step 2:** Run Bellman-Ford from `s` to detect negative cycles, using 2D array like done in Bellman-Ford
- If any vertex has distance -∞, original graph has negative cycle → ABORT
- Otherwise, we get distances δ(s,v) for all vertices v

### Phase 2: Reweight Edges Using Potential Function

**Potential Function:** h(v) = δ(s,v) for all vertices v

**Reweighting Formula:** For each edge (u,v):
```
new_weight(u,v) = old_weight(u,v) + h(u) - h(v)
```

**Mathematical Guarantee:** This makes all edge weights non-negative!

### Phase 3: Run Dijkstra from Each Vertex

**Step 1:** Run Dijkstra from each vertex in reweighted graph G'
**Step 2:** Convert distances back to original graph:
```
original_distance(u,v) = reweighted_distance(u,v) - h(u) + h(v)
```


![pic](./09-1.png)

![pic](./09-2.png)

![pic](./09-3.png)

![pic](./09-4.png)

![pic](./09-5.png)

![pic](./09-6.png)

![pic](./09-7.png)

![pic](./09-8.png)

![pic](./09-9.png)

![pic](./09-10.png)




## 💻 Implementation

### Complete Johnson's Algorithm
```python
def johnsons_algorithm(graph):
    """
    Johnson's algorithm for all-pairs shortest paths
    
    Args:
        graph: dict where graph[u] = [(v, weight), ...]
    
    Returns:
        all_distances: dict where all_distances[u][v] = shortest distance u→v
        or None if negative cycle detected
    """
    vertices = list(graph.keys())
    
    # Phase 1: Add super source and run Bellman-Ford
    super_source = 'SUPER_SOURCE'
    extended_graph = graph.copy()
    extended_graph[super_source] = [(v, 0) for v in vertices]
    
    # Run Bellman-Ford from super source
    distances_from_super, has_negative_cycle = bellman_ford(extended_graph, super_source)
    
    if has_negative_cycle:
        return None  # Negative cycle detected
    
    # Phase 2: Reweight edges using potential function
    h = distances_from_super  # Potential function
    reweighted_graph = {}
    
    for u in vertices:
        reweighted_graph[u] = []
        for v, weight in graph[u]:
            new_weight = weight + h[u] - h[v]
            reweighted_graph[u].append((v, new_weight))
    
    # Phase 3: Run Dijkstra from each vertex
    all_distances = {}
    for u in vertices:
        reweighted_distances = dijkstra(reweighted_graph, u)
        
        # Convert back to original distances
        all_distances[u] = {}
        for v in vertices:
            if reweighted_distances[v] != float('inf'):
                all_distances[u][v] = reweighted_distances[v] - h[u] + h[v]
            else:
                all_distances[u][v] = float('inf')
    
    return all_distances

def bellman_ford(graph, start):
    """Bellman-Ford with negative cycle detection"""
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0
    
    # Relax edges V-1 times
    for _ in range(len(graph) - 1):
        for u in graph:
            if distances[u] != float('inf'):
                for v, weight in graph[u]:
                    if distances[u] + weight < distances[v]:
                        distances[v] = distances[u] + weight
    
    # Check for negative cycles
    for u in graph:
        if distances[u] != float('inf'):
            for v, weight in graph[u]:
                if distances[u] + weight < distances[v]:
                    return distances, True  # Negative cycle found
    
    return distances, False

def dijkstra(graph, start):
    """Standard Dijkstra implementation"""
    import heapq
    
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_dist, u = heapq.heappop(pq)
        
        if u in visited:
            continue
        visited.add(u)
        
        for v, weight in graph[u]:
            if v not in visited:
                new_dist = current_dist + weight
                if new_dist < distances[v]:
                    distances[v] = new_dist
                    heapq.heappush(pq, (new_dist, v))
    
    return distances
```

### Example Usage
```python
# Example graph with negative weights
graph = {
    'A': [('B', 4), ('C', 2)],
    'B': [('C', -5), ('D', 6)],  # Negative weight!
    'C': [('D', 3)],
    'D': []
}

result = johnsons_algorithm(graph)
if result:
    print("All-pairs shortest distances:")
    for u in result:
        for v in result[u]:
            print(f"{u} → {v}: {result[u][v]}")
else:
    print("Negative cycle detected!")
```

## 📝 Visual Examples

### Example 1: Graph Transformation Process
```
[INSERT YOUR PICTURE 1 HERE: Original graph with mixed positive/negative weights]
```

### Example 2: Adding Super Source
```
[INSERT YOUR PICTURE 2 HERE: Graph with super source added and 0-weight edges]
```

### Example 3: Reweighting Process
```
[INSERT YOUR PICTURE 3 HERE: Before and after reweighting showing all positive weights]
```

### Example 4: Final Result
```
[INSERT YOUR PICTURE 4 HERE: All-pairs shortest path matrix/table]
```

## 🆚 Algorithm Comparison

### When to Use Each APSP Algorithm:

| Graph Type | Algorithm | Time Complexity | Space | Best For |
|------------|-----------|-----------------|-------|----------|
| **Dense graphs** | Floyd-Warshall | O(V³) | O(V²) | Simple implementation |
| **Sparse + non-negative** | Dijkstra × V | O(V²log V + VE) | O(V²) | GPS networks |
| **Sparse + negative weights** | **Johnson's** | **O(V²log V + VE)** | **O(V²)** | **General sparse graphs** |
| **Dense + negative weights** | Floyd-Warshall | O(V³) | O(V²) | When V³ ≈ V²log V + VE |

### Decision Tree:
```
Need all-pairs shortest paths?
├── Dense graph (E ≈ V²) → Floyd-Warshall
└── Sparse graph (E ≪ V²) → 
    ├── Non-negative weights → Dijkstra × V times
    └── Negative weights possible → Johnson's Algorithm
```

## 🌍 Real-World Applications

### 1. Supply Chain Optimization 📦
- **Vertices:** Warehouses, distribution centers
- **Edges:** Shipping costs/credits (bulk discounts = negative weights)
- **Goal:** Find optimal routes between all facility pairs
- **Johnson's advantage:** Handles complex pricing with discounts

### 2. Financial Network Analysis 💰
- **Vertices:** Financial institutions
- **Edges:** Transaction costs/rewards
- **Goal:** Find arbitrage opportunities across all pairs
- **Johnson's advantage:** Detects negative cycles (arbitrage loops)

### 3. Communication Network Routing 📡
- **Vertices:** Network nodes/routers
- **Edges:** Latency costs/QoS bonuses
- **Goal:** Optimal routing tables for all node pairs
- **Johnson's advantage:** Efficient for sparse network topologies

### 4. Transportation Planning 🚗
- **Vertices:** Cities/stations
- **Edges:** Travel costs/toll rebates
- **Goal:** Complete distance matrix for trip planning
- **Johnson's advantage:** Handles subsidized routes efficiently

## 📊 Mathematical Foundations

### The Potential Function Theory

**Core Theorem:** If h: V → ℝ is any function, then reweighting edges as:
$$w'(u,v) = w(u,v) + h(u) - h(v)$$
preserves shortest paths between all vertex pairs.

**Proof:** For any path π = (v₀, v₁, ..., vₖ):
$$w'(\pi) = \sum_{i=1}^{k} w'(v_{i-1}, v_i) = \sum_{i=1}^{k} [w(v_{i-1}, v_i) + h(v_{i-1}) - h(v_i)]$$
$$= w(\pi) + h(v_0) - h(v_k)$$

**Key insight:** All paths from v₀ to vₖ change by the same constant h(v₀) - h(v₋), so relative ordering (shortest path) is preserved.

### Non-negative Weight Guarantee

**Triangle Inequality Condition:** We need h such that:
$$w(u,v) + h(u) - h(v) \geq 0 \text{ for all edges } (u,v)$$

**Rearranging:** $h(v) \leq h(u) + w(u,v)$ for all edges (u,v)

**Johnson's Solution:** Use h(v) = δ(s,v) where s is super source with 0-weight edges to all vertices.

**Why this works:** 
- δ(s,v) ≤ δ(s,u) + w(u,v) by triangle inequality
- Rearranging: δ(s,v) - δ(s,u) ≤ w(u,v)
- Therefore: w(u,v) + δ(s,u) - δ(s,v) ≥ 0 ✓

### Negative Cycle Detection

**Theorem:** If Bellman-Ford finds δ(s,v) = -∞ for any vertex v, then the original graph contains a negative cycle.

**Proof:** 
- Super source s has no incoming edges, so adding it cannot create new cycles
- If reweighted graph has negative cycle, it must use only original edges
- Therefore, original graph contained the negative cycle

### Time Complexity Analysis

**Phase 1: Bellman-Ford**
- Graph construction: O(V + E)
- Bellman-Ford execution: O(VE)
- **Subtotal:** O(VE)

**Phase 2: Reweighting**
- Edge reweighting: O(E)
- **Subtotal:** O(E)

**Phase 3: Multiple Dijkstra**
- V runs of Dijkstra: V × O(V log V + E)
- Distance conversion: O(V²)
- **Subtotal:** O(V²log V + VE)

**Total Time Complexity:** O(VE + V²log V + VE) = **O(V²log V + VE)**

**Space Complexity:** O(V²) for storing all-pairs distances

### Comparison with Alternatives

**Johnson's vs Floyd-Warshall:**
- **Johnson's:** O(V²log V + VE) - better for sparse graphs (E ≪ V²)
- **Floyd-Warshall:** O(V³) - simpler but slower for sparse graphs
- **Break-even point:** When E ≈ V²/log V

**Johnson's vs Repeated Bellman-Ford:**
- **Johnson's:** O(V²log V + VE)
- **Bellman-Ford × V:** O(V²E)
- **Johnson's advantage:** Always better when E = Ω(V log V)

## ⚡ Advanced Topics

### Optimizations and Variations

**Early Termination in Dijkstra:**
- Stop Dijkstra early if only specific targets needed
- Useful for sparse distance matrices

**Bidirectional Johnson's:**
- Run algorithm in both directions
- Can reduce constants in practice

**Parallel Implementation:**
- Phase 3 (multiple Dijkstra) is embarrassingly parallel
- Each Dijkstra run is independent

### Space Optimizations

**On-demand Distance Computation:**
```python
def get_distance_on_demand(u, v, h, graph):
    """Compute single distance without storing full matrix"""
    dist_reweighted = dijkstra_single_target(reweighted_graph, u, v)
    return dist_reweighted - h[u] + h[v]
```

**Compressed Representation:**
- Store only potential function h and run Dijkstra on-demand
- Trade time for space: O(V) space vs O(V²)

## ⚠️ Common Mistakes & Best Practices

### ❌ What NOT to do:

1. **Forgetting negative cycle check** - Algorithm assumes no negative cycles
2. **Incorrect distance conversion** - Must subtract h(u) and add h(v)
3. **Using with positive weights only** - Just use Dijkstra × V times
4. **Applying to dense graphs** - Floyd-Warshall is simpler and often faster

## 🎯 Summary

### Use Johnson's Algorithm when:
- ✅ Need all-pairs shortest paths
- ✅ Graph has negative edge weights
- ✅ Graph is sparse (E ≪ V²)
- ✅ Want better than O(V³) performance



### Final Complexity:
**Time:** O(V²log V + VE) - optimal for sparse graphs with negative weights  
**Space:** O(V²) - stores complete distance matrix

**Bottom Line:** Johnson's algorithm is the perfect solution when you need all-pairs shortest paths in sparse graphs with negative weights - it's like having the best of both Bellman-Ford (handles negatives) and Dijkstra (fast execution)!